In [15]:
import os
import numpy as np
import pandas as pd
import zipfile
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from keras.regularizers import l2
from PIL import UnidentifiedImageError
import tensorflow
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten, Dropout
from glob import glob
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image
import random
import shutil

In [ ]:
import tensorflow as tf
print(tf.__version__)

2.17.0


In [ ]:
!pwd

/content


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
zip_path = "/content/drive/MyDrive/Pizza Image Classification/Data.zip"
extract_path = "/content/drive/MyDrive/Pizza Image Classification"

In [ ]:
# Unzipping the file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

In [4]:
# Path to your extracted dataset in Google Drive
dataset_path = '/content/drive/MyDrive/Pizza Image Classification/Data'

# Define the class folders
pizza_folder = os.path.join(dataset_path, 'pizza')
not_pizza_folder = os.path.join(dataset_path, 'not_pizza')

# Define destination paths for training and validation sets
data_path = '/content/drive/MyDrive/Pizza Image Classification'
train_dir = os.path.join(data_path, 'train')
val_dir = os.path.join(data_path, 'val')

In [5]:
# Create directories if they do not exist
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)
os.makedirs(os.path.join(train_dir, 'Pizza'), exist_ok=True)
os.makedirs(os.path.join(train_dir, 'Not pizza'), exist_ok=True)
os.makedirs(os.path.join(val_dir, 'Pizza'), exist_ok=True)
os.makedirs(os.path.join(val_dir, 'Not pizza'), exist_ok=True)

In [ ]:
# Helper function to split data
def split_data(source_dir, train_dir, val_dir, class_name, test_size=0.2):
    # List all files in the source directory
    all_files = os.listdir(source_dir)

    # Get file paths
    file_paths = [os.path.join(source_dir, f) for f in all_files if os.path.isfile(os.path.join(source_dir, f))]

    # Split the data into training and validation sets
    train_files, val_files = train_test_split(file_paths, test_size=test_size, random_state=42)

    # Copy files to the respective directories
    for file_path in train_files:
        shutil.copy(file_path, os.path.join(train_dir, class_name))

    for file_path in val_files:
        shutil.copy(file_path, os.path.join(val_dir, class_name))

# Split data for each class
split_data(pizza_folder, train_dir, val_dir, 'Pizza')
split_data(not_pizza_folder, train_dir, val_dir, 'Not pizza')

print('Data has been successfully split into training and validation sets!')

Data has been successfully split into training and validation sets!


In [ ]:
# Function to count files in each directory
def count_files(directory):
    count = 0
    for root, dirs, files in os.walk(directory):
        count += len(files)
    return count

# Print counts for each set
print(f"Training set - Pizza: {count_files(os.path.join(train_dir, 'Pizza'))} images")
print(f"Training set - Not pizza: {count_files(os.path.join(train_dir, 'Not pizza'))} images")
print(f"Validation set - Pizza: {count_files(os.path.join(val_dir, 'Pizza'))} images")
print(f"Validation set - Not pizza: {count_files(os.path.join(val_dir, 'Not pizza'))} images")


Training set - Pizza: 400 images
Training set - Not pizza: 399 images
Validation set - Pizza: 100 images
Validation set - Not pizza: 100 images


In [ ]:
def display_random_images(image_dir, class_name, num_images=5):
    """
    Display random images from the specified directory.

    Args:
        image_dir (str): Path to the directory containing images.
        class_name (str): The class name (Pizza or Not pizza).
        num_images (int): Number of random images to display.
    """
    # Get list of all images in the directory
    all_images = os.listdir(image_dir)

    # Select a random subset of images
    random_images = random.sample(all_images, min(len(all_images), num_images))

    # Plot the random images
    plt.figure(figsize=(12, 6))
    for i, img_name in enumerate(random_images):
        img_path = os.path.join(image_dir, img_name)
        img = mpimg.imread(img_path)
        plt.subplot(1, num_images, i + 1)
        plt.imshow(img)
        plt.title(f"{class_name} - {img_name}")
        plt.axis('off')

    plt.show()


In [ ]:
# Paths to the training and validation directories
train_pizza_dir = '/content/drive/MyDrive/Pizza Image Classification/train/Pizza'
train_not_pizza_dir = '/content/drive/MyDrive/Pizza Image Classification/train/Not pizza'
val_pizza_dir = '/content/drive/MyDrive/Pizza Image Classification/val/Pizza'
val_not_pizza_dir = '/content/drive/MyDrive/Pizza Image Classification/val/Not pizza'

# Display random images from each set
print("Training Set - Pizza")
display_random_images(train_pizza_dir, 'Pizza', num_images=3)

print("Training Set - Not pizza")
display_random_images(train_not_pizza_dir, 'Not pizza', num_images=3)

print("Validation Set - Pizza")
display_random_images(val_pizza_dir, 'Pizza', num_images=3)

print("Validation Set - Not pizza")
display_random_images(val_not_pizza_dir, 'Not pizza', num_images=3)


In [ ]:
# To check if the images are in the correct fromat.
def check_images(directory):
    for root, _, files in os.walk(directory):
        for file in files:
            try:
                img_path = os.path.join(root, file)
                img = Image.open(img_path)
                img.verify()  # Check if image can be opened
            except (IOError, SyntaxError) as e:
                print(f"Corrupted or invalid image file: {img_path}")

# Check both the training and validation directories
check_images('/content/drive/MyDrive/Pizza Image Classification/train')
check_images('/content/drive/MyDrive/Pizza Image Classification/val')

In [6]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

val_datagen = ImageDataGenerator(rescale=1./255)

# Load the images from the training directory
train_dir = '/content/drive/MyDrive/Pizza Image Classification/train'
val_dir = '/content/drive/MyDrive/Pizza Image Classification/val'

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=True
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

Found 799 images belonging to 2 classes.
Found 200 images belonging to 2 classes.


In [ ]:
# Load the VGG16 model with pretrained weights and exclude the top classification layers
vgg16 = VGG16(input_shape=(224, 224, 3), weights='imagenet', include_top=False)

# Freeze the layers of the VGG16 model
for layer in vgg16.layers:
    layer.trainable = False

# Define input tensor
inputs = vgg16.input

# Add Flatten layer to convert 2D outputs to 1D
x = Flatten()(vgg16.output)

# Add Dense layer with L2 regularization and ReLU activation
x = Dense(256, activation='relu', kernel_regularizer=l2(0.01))(x)

# Add Dropout layer to prevent overfitting
x = Dropout(0.5)(x)

# Add output Dense layer for binary classification with softmax
outputs = Dense(2, activation='softmax', kernel_regularizer=l2(0.01))(x)

# Create the model
model = Model(inputs=inputs, outputs=outputs)

# Compile the model with Adam optimizer and categorical cross-entropy loss
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print model summary to check the architecture
model.summary()

In [7]:
# Calculate steps per epoch
steps_per_epoch = train_generator.samples // train_generator.batch_size
validation_steps = val_generator.samples // val_generator.batch_size

In [ ]:
# Add EarlyStopping to stop training when the validation loss does not improve
earlystopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# fit the model
r = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=30,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
    callbacks=[earlystopping]  # Include early stopping callback
)

In [ ]:
## MODEL 2 VGG19

# Load the VGG19 model with pretrained weights and exclude the top classification layers
vgg19 = VGG19(input_shape=(224, 224, 3), weights='imagenet', include_top=False)

# Freeze the layers of the VGG19 model
for layer in vgg19.layers:
    layer.trainable = False

# Define input tensor
inputs = vgg19.input

# Add Flatten layer to convert 2D outputs to 1D
x = Flatten()(vgg19.output)

# Add Dense layer with L2 regularization and ReLU activation
x = Dense(256, activation='relu', kernel_regularizer=l2(0.01))(x)

# Add Dropout layer to prevent overfitting
x = Dropout(0.5)(x)

# Add output Dense layer for binary classification with softmax
outputs = Dense(2, activation='softmax', kernel_regularizer=l2(0.01))(x)

# Create the model
model2 = Model(inputs=inputs, outputs=outputs)

# Compile the model with Adam optimizer and categorical cross-entropy loss
model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print model summary to check the architecture
model2.summary()

In [9]:
# Set early stopping callback to monitor validation loss
early_stopping = EarlyStopping(
    monitor='val_loss',        # Monitor the validation loss
    patience=3,                # Stop training after 3 epochs without improvement
    restore_best_weights=True  # Restore model weights from the epoch with the best validation loss
)

In [ ]:
history = model2.fit(
    train_generator,
    validation_data=val_generator,
    epochs=30,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
    callbacks=[early_stopping]  # Include early stopping callback
)

# Evaluate the model
loss, accuracy = model2.evaluate(val_generator)
print(f'Validation Loss: {loss}')
print(f'Validation Accuracy: {accuracy}')

In [ ]:
## Model RESNET50

# Define the ResNet model architecture in a function
resnet = ResNet50(input_shape=(224, 224, 3), weights='imagenet', include_top=False)

# Freeze the layers of ResNet
for layer in resnet.layers:
    layer.trainable = False

inputs = resnet.input
x = Flatten()(resnet.output)

# Add dense layer with regularization and dropout
x = Dense(256, activation='relu', kernel_regularizer=l2(0.01))(x)
x = Dropout(0.5)(x)

outputs = Dense(2, activation='softmax', kernel_regularizer=l2(0.01))(x)

# Create the model
model3 = Model(inputs=inputs, outputs=outputs)

# Compile the model
model3.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print model summary to check the architecture
model3.summary()

In [ ]:
# Define EarlyStopping callback
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Fit the model
history = model3.fit(
    train_generator,
    validation_data=val_generator,
    epochs=20,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
    callbacks=[early_stop],  # Add the early stopping callback here
    verbose=1
)

In [ ]:
# Define the directory path
model_dir = '/content/drive/MyDrive/Pizza Image Classification/Model'

# Create the directory if it doesn't exist
os.makedirs(model_dir, exist_ok=True)

# Save the model in the "Model" folder
model2.save(os.path.join(model_dir, 'vgg19_model.h5'))

print(f"Model saved at {os.path.join(model_dir, 'vgg19_model.h5')}")

In [23]:
# Load the saved model from the directory
model_path = '/content/drive/MyDrive/Pizza Image Classification/Model/vgg19_model.h5'
Classifier = load_model(model_path)

print("Model loaded successfully!")


Model loaded successfully!


In [ ]:
# Define the path to your folder with images
image_folder = '/content/drive/MyDrive/Pizza Image Classification/Modeltrial'


# Map class indices to labels (update based on your actual classes)
class_labels = {1: 'Pizza', 0: 'Not Pizza'}

# Iterate over every image in the folder
for img_file in os.listdir(image_folder):
    # Construct the full image path
    img_path = os.path.join(image_folder, img_file)

    # Load and preprocess the image
    img = image.load_img(img_path, target_size=(224, 224))  # Resize the image to 224x224
    img_array = image.img_to_array(img)  # Convert the image to array
    img_array = np.expand_dims(img_array, axis=0)  # Add a batch dimension (1, 224, 224, 3)
    img_array /= 255.0  # Normalize the image

    # Make predictions using the loaded model
    predictions = Classifier.predict(img_array)

    # Get the predicted class
    predicted_class = np.argmax(predictions, axis=-1)

    # Print the result for each image
    print(f"Image: {img_file} -> Predicted class: {class_labels[predicted_class[0]]}")
